# Route Planner - Group 17 Tiger Air
### Team Members
 - Abhishek Sinha
 - Alan Young
 - Colin Beagan
 - Nimisha D B Raju
 
This notebook is for project demonstration purposes. More detail will be included in each team members individual notebooks

In [1]:
# import required modules
import math
# for permutation function
import itertools
from collections import Counter
from random import choice
import pandas as pd
import copy
import csv

### Read in CSV files

In [2]:
airport_df=pd.read_csv('airport.csv')
currencyRates_df=pd.read_csv('currencyrates.csv')
countryCurrency_df=pd.read_csv('countrycurrency.csv')
aircraft_df=pd.read_csv('aircraft.csv')
test_df=pd.read_csv('test.csv', header=None)

### Handle Currency data

In [3]:
#Currency data is spread across the three CSV files. We want to get "toEuro" value in airport dataframe. 
#Requires two joins. Joining will remove airports whose country not have a currency in the "currencyRates" csv...
# 1. Rename columns to allow join operation.
countryCurrency_df.rename(columns={'currency_alphabetic_code':'CurrencyCode','name':'COUNTRY'}, inplace=True)
# 2. Join "countryCurrency" data to "currencyRates"
currencyRates_df = pd.merge(currencyRates_df[['CurrencyCode','toEuro']], countryCurrency_df[['CurrencyCode','COUNTRY']], how='inner', on='CurrencyCode')
# 3. Join "currencyRates" to "airport"
airport_df = pd.merge(airport_df[['NAME', 'IATA', 'Lat', 'Log', 'COUNTRY']], currencyRates_df[['toEuro','COUNTRY']], how='inner', on='COUNTRY')

### Setup AirportAtlas Class

In [4]:
class AirportAtlas:
    airports = []
    airportsNeeded = Counter()
    airportsCount = 0
    
    def __init__(self, airportsCSV, inputCSV):
        # exclude last column of input CSV(aircraft)
        self.setNeededAirports(inputCSV)
        self.instantiateAirports(airportsCSV)
    
    # pass an airport code to fetch all airport details
    @classmethod
    def getAirport(cls, airportCode):
        for i in range(cls.airportsCount):
            if(cls.airports[i].getCode() == airportCode):
                return cls.airports[i]
        return None
    
    # pass an airport code to fetch the airport name
    @classmethod
    def getAirportName(cls, airportCode):
        for i in range(cls.airportsCount):
            if(cls.airports[i].getCode() == airportCode):
                return cls.airports[i].getName()
        return None
    
    # pass an aircraft code to remove aircraft from aircraft atlas
    @classmethod
    def removeAirport(cls, code):
        #print("in airport atlas remove airport method: " + code)
        #for i in range(cls.airportsCount):
            #if(cls.airports[i].getCode() == code):
        cls.airports.remove(cls.getAirport(code))
        cls.airportsCount -= 1
      
    @classmethod
    def contains(cls, airportCode):
        for i in range(cls.airportsCount):
            if(cls.airports[i].getCode() == airportCode):
                return True
        return False
    
    @classmethod
    def searchCountry(cls, country):
        for i in range(cls.airportsCount):
            if(cls.airports[i].isCountry(country)):
                cls.airports[i].printDetails()
        
    def setNeededAirports(self, data):
        for i, row in data.iterrows():
            #print("in row", i)
            for j in range(len(row) -1):
                self.airportsNeeded[row[j]] += 1
        #print(self.airportsNeeded)
                #if row[j] in self.airportsNeeded:
                #    self.airportsNeeded(row[j]) += 1
                #else:
                #    self.airportsNeeded.append(row[j])
        
    def instantiateAirports(self, data):
        for i, row in data.iterrows(): 
            if self.isAirportNeeded(row['IATA']):
                if self.contains(row['IATA']) == False:
                    self.addAirport(row['NAME'], row['IATA'], row['Lat'], row['Log'], row['COUNTRY'], row['toEuro'], self.airportsNeeded[row['IATA']])
                #for j in range(self.airportsNeeded[row['IATA']], 1, -1):
                   # self.getAirport(row['IATA']).increaseOccurances()
                    
                    #print(row['toEuro'])
        
    def isAirportNeeded(self, airportCode):
        return airportCode in self.airportsNeeded
        
    def addAirport(self, name, code, lat, lng, country, exchangeRate, occurances):
        AirportAtlas.airportsCount += 1
        AirportAtlas.airports.append(Airport(name, code, lat, lng, country, exchangeRate, occurances))
    
    def printAirports(self):    
        for port in AirportAtlas.airports:
            port.printDetails()
            

### Setup Airport Class

In [5]:
class Airport:
    
#     #not needed now as join is working...
#     currencyLookup = {}
#     for i, row in currencyRates_df.iterrows():
#         currencyLookup[row['COUNTRY']] = row['toEuro']
    
    def __init__(self, name, code, lat, lng, country, exchangeRate, occurances = 1):
        self.remainingOccurances = occurances
        self.setName(name)
        self.setCode(code)
        self.setLat(lat)
        self.setLng(lng)
        self.setCountry(country)
        self.setCurrencyMultiplier(exchangeRate)
        
    def printDetails(self):
        print(str(self.getName()) + " <--> " +
              str(self.getCode()) + " <--> " +
              str(self.getLat()) + " <--> " +
              str(self.getLng()) + " <--> " +
              str(self.getCountry()) + " <--> " +
              str(self.getCurrencyMultiplier()))
        
    def isCountry(self, country):
        return self.getCountry() == country
        
    def increaseOccurances(self, amount = 1):
        self.remainingOccurances += amount
        
    def decreaseOccurances(self):
        self.remainingOccurances -= 1
        if self.remainingOccurances <= 0:
            AirportAtlas.removeAirport(self.getCode())
        
    def setName(self, name):
        self.name = name
        
    def setCode(self, code):
        self.code = code
        
    def setLat(self, latitude):
        self.latitude = latitude
        
    def setLng(self, longitude):
        self.longitude = longitude
        
    def setCountry(self, country):
        self.country = country
        
    def setCurrencyMultiplier(self, exchangeRate):
        # to account for countries with no currency data. Set dafault to 1....
        try:
            self.currencyMultiplier = exchangeRate
        except:
            self.currencyMultiplier = 1
            
    def getName(self):
        return self.name
        
    def getCode(self):
        return self.code
        
    def getLat(self):
        return self.latitude
        
    def getLng(self):
        return self.longitude
    
    def getCountry(self):
        return self.country  
    
    def getCurrencyMultiplier(self):
        return self.currencyMultiplier

### Setup AircraftAtlas Class

In [6]:
class AircraftAtlas:
    aircrafts = []
    aircraftsNeeded = Counter()
    aircraftsCount = 0
    
    def __init__(self, aircraftsCSV, inputCSV):
        self.setNeededAircrafts(inputCSV)
        self.instantiateAircrafts(aircraftsCSV)
        
    @classmethod
    def getAircraft(cls, aircraftCode):
        for i in range(cls.aircraftsCount):
            if(cls.aircrafts[i].getCode() == aircraftCode):
                return cls.aircrafts[i]
        return None
        
    @classmethod
    def searchManufacturer(cls, manufacturer):
        for i in range(cls.aircraftsCount):
            if(cls.aircrafts[i].isManufacturedBy(manufacturer)):
                cls.aicrafts[i].printDetails()
       
    @classmethod
    def contains(cls, aircraftCode):
        for i in range(cls.aircraftsCount):
            if(cls.aircrafts[i].getCode() == aircraftCode):
                return True
        return False
    
    @classmethod
    def removeAircraft(cls, code):
        #print("in aircraft atlas remove aircraft method: " + code)
        #for i in range(cls.aircraftsCount):
            #if(cls.aircrafts[i].getCode() == code):
        cls.aircrafts.remove(cls.getAircraft(code))
        cls.aircraftsCount -= 1
    
    def setNeededAircrafts(self, data):
        for i, row in data.iterrows():  
            self.aircraftsNeeded[row[len(row) - 1]] += 1
        #print(self.aircraftsNeeded)
    
    def instantiateAircrafts(self, data):
        for i, row in data.iterrows():  
            if self.isAircraftNeeded(row['code']):
                if self.contains(row['code']) == False:
                    self.addAircraft(row['code'], row['units'], row['manufacturer'], row['range'], self.aircraftsNeeded[row['code']])
    
    def isAircraftNeeded(self, aircraftCode):
        return aircraftCode in self.aircraftsNeeded
    
    def addAircraft(self, code, units, manufacturer, flightRange, occurances):
        AircraftAtlas.aircraftsCount += 1
        AircraftAtlas.aircrafts.append(Aircraft(code, units, manufacturer, flightRange, occurances))
        
    def printAircrafts(self):    
        for craft in AircraftAtlas.aircrafts:
            craft.printDetails()
            

### Setup Aircraft Class

In [7]:
class Aircraft:
    def __init__(self, code, units, manu, dist, occurances = 1):
        self.remainingOccurances = occurances
        self.rangeModifier = 1
        self.setCode(code)
        self.setUnits(units)
        self.setManufacturer(manu)
        self.setFlightRange(dist)
        self.route = None
       
    def printDetails(self):
        print(str(self.getCode()) + " <--> " + str(self.getManufacturer()) + " <--> " + str(self.getUnits()) + " <--> " + str(self.getFlightRange()))
    
    def isManufacturedBy(self, manufacturer):
        return self.getManufacturer() == manufacturer
    
    def increaseOccurances(self, amount = 1):
        self.remainingOccurances += amount
        
    def decreaseOccurances(self):
        self.remainingOccurances -= 1
        if self.remainingOccurances <= 0:
            AircraftAtlas.removeAircraft(self.getCode())
    
    def setCode(self, code):
        self.code = code
        
    def setUnits(self, units):
        self.units = units
        #print(self.units)
        #if the units were given in miles, adjust the range modifier
        if(self.units == "imperial"):
            self.rangeModifier = 1.6093
        
    def setManufacturer(self, manu):
        self.manufacturer = manu
        
    def setFlightRange(self, dist):
        self.flightRange = dist * self.rangeModifier
        
    def getCode(self):
        return self.code
        
    def getUnits(self):
        return self.units
        
    def getManufacturer(self):
        return self.manufacturer
        
    def getFlightRange(self):
        return self.flightRange
    
    #instantiate a Route object with the airport codes to be visited by this aircraft
    def setRoute(self, routeNames):
        self.route = Route(routeNames)
        self.printRouteReport(routeNames)
        self.decreaseOccurances()
        
    def printRouteReport(self, routeNames):
        self.route.printReport(routeNames)

### Setup Route Class

In [8]:
class Route:
    """Route class to calculate cheapest route from given list of airport names
    Class will store attributes relating to results to route calculations"""
    
    # class variable to store cached results
    final={}
    
    def __init__(self, Codes):
        self.__input = Codes
        self.__startAirport = Codes[:1]
        self.__endAirport = self.__startAirport
        self.__airportObjects = []  # list of airportNames codes for the route
        self.__airportNames = []  # list of airport name for the route
        self.__airportCodes = Codes[:-1]
        self.__aircraftObject = []
        self.__cheapestRoute = ""
        self.__cheapestRouteCost = ""
        self.__impossibleLegs = set()
        self.setAirports(Codes[:-1])  # setup airportObject/names using all but last item in "Codes" 
        self.setAircraft(Codes[-1])  # set the last item in list to be the aircraft code
        self.__maxRange = self.__aircraftObject.getFlightRange()
        self.__distTable = {}
        self.generateDistanceTable()
        self.__costTable = {}
        self.generateCostTable()
        self.__permutationList = []
        self.generatePermutations()
        self.CheapestRouteCalculator(self.__permutationList)


    def setAirports(self, airportCodes):
        """Method to fetch the airportObjects and Names 
        from the AirportAtlas and store as instance attributes"""

        # loop through the airport codes provided, fetch airportObject/Names from AirportAtlas
        for code in airportCodes:
            self.__airportObjects.append(AirportAtlas.getAirport(code))
            self.__airportNames.append(AirportAtlas.getAirportName(code))

    def setAircraft(self, aircraftCode):
        """Method to fetch the aircraftObject from the from 
        the AircraftAtlas and store as instance attribute"""

        # fetch the aircraftObject from the AirportAtlas 
        self.__aircraftObject = AircraftAtlas.getAircraft(aircraftCode)

    def printAirportInformation(self):
        print("6. Airports along this route;")
        print("======================================================")
        for airport in self.__airportObjects:
            airport.printDetails()
        print("======================================================\n")

    def printAircraftInformation(self):
        print("5. Aircraft details")
        print("======================================================")
        self.__aircraftObject.printDetails()
        print("======================================================\n")

    def generateDistanceTable(self):
        """Method to generate Distance lookup table
        Assigns result to instance attribute distTable"""
        distances = {}

        for src_airport in self.__airportObjects:
            distances[src_airport.getCode()] = {}
            for dest_airport in self.__airportObjects:
                leg_distance = self.greatCircleDistance2(src_airport, dest_airport)
                # print(src_airport.getCode(), dest_airport.getCode(), leg_distance)
                distances[src_airport.getCode()][dest_airport.getCode()] = leg_distance
                # check if plane range long enough
                if self.__maxRange < leg_distance:
                    # use frosen sets to allow storing of nested sets - we do not want to add in the same airport in opposite direction
                    impossibleLeg = frozenset([src_airport.getCode(), dest_airport.getCode()])
                    self.__impossibleLegs.add(impossibleLeg)
        # print(distances)
        self.__distTable = distances

    def printDistanceTable(self):
        print("3. Distance Table (km)")
        print("======================================================")
        distTable = pd.DataFrame(self.__distTable).sort_index(axis=1)
        print(distTable)
        print("======================================================\n")

    def generateCostTable(self):
        """Method to generate Cost lookup table
        Assigns result to instance attribute costTable"""

        # used Distance table and multiplies each entry by corresponding currency multiplier
        journeyCosts = copy.deepcopy(self.__distTable)

        for src_airport in journeyCosts:
            for dest_airport in journeyCosts[src_airport]:
                journeyCosts[src_airport][dest_airport] *= AirportAtlas.getAirport(src_airport).getCurrencyMultiplier()

        self.__costTable = journeyCosts

    def printCostTable(self):
        print("4. Cost Table (€)")
        print("======================================================")
        costTable = pd.DataFrame(self.__costTable).sort_index(axis=1).round(2)
        print(costTable)
        print("======================================================\n")

    def greatCircleDistance2(self, source, destination):
        '''Function to calculate the distance between to points using the
        great circle. Function arguments to be passed as coordinate list
        Location coordinates stored in location dictionary.
        Function to return distance in km to 2 decimal places'''

        # mean earth radius (earth is a flattened sphere)
        radius = 6371

        # set coordinates
        x1 = source.getLat()
        y1 = source.getLng()
        x2 = destination.getLat()
        y2 = destination.getLng()

        # convert to radians for centralAngle formula
        x1 = math.radians(x1)
        y1 = math.radians(y1)
        x2 = math.radians(x2)
        y2 = math.radians(y2)

        # angle between the two airportNames in radians
        a = math.sin(x1) * math.sin(x2) + math.cos(x1) * math.cos(x2) * math.cos(y1 - y2)
        # round to avoid floating point error
        centralAngle = math.acos(round(a, 5))

        # arc length = radius * angle(in radians)
        arcLength = radius * centralAngle

        return round(arcLength, 2)

    def journeyCost2(self, source, destination):
        '''Function to calculate the journey costs between to points using the
        great circle function. Fuel cost based on departure airport
        Function to return cost in EUR to 2 decimal places'''

        return round(self.greatCircleDistance2(source, destination) * source.getCurrencyMultiplier(), 2)

    def generatePermutations(self):
        """Method to generate all valid route permutations. Permutations will 
        be generated and checked for validity against maximum flight range. Any
        permutations that are invalid will be removed from permutation list"""

        start_end_airport = self.__airportCodes.pop(0)
        permu = list(itertools.permutations(self.__airportCodes))

        # print("\npermutations are\n", permu)

        # convert to list of lists first to allow modification
        permu_list = []
        for i in range(len(permu)):
            permu_list.append(list(permu[i]))

        # add add start and end back to permutations for calculating cost
        for i in range(len(permu)):
            permu_list[i].insert(0, start_end_airport)
            permu_list[i].append(start_end_airport)
        # print("\npermutations\n", permu_list)

        invalid_route_indexes = []
        for i in range(len(permu_list)):
            for j in range(len(permu_list[i]) - 1):
                # if any one flight distance > maxRange then not valid permutaion
                if self.__distTable[permu_list[i][j]][permu_list[i][j + 1]] > self.__maxRange:
                    invalid_route_indexes.append(i)
                    #                     print("For route: ",permu_list[i])
                    #                     print("found an invalid route",permu_list[i][j], permu_list[i][j+1], self.distTable[permu_list[i][j]][permu_list[i][j+1]])
                    break

        # print("invalid route indexes", invalid_route_indexes)

        for index in sorted(invalid_route_indexes, reverse=True):
            del permu_list[index]
        # print("remaining valid permutations",permu_list)

        self.__permutationList = permu_list

        
    def CheapestRouteCalculator(self, permu_list):
        """Method to calculate the cheapest route based on permutation list 
        and set instance attributes for cheapest route and cheapest route cost"""

        # if number of permutations < 1 then no valid route exists
        if len(permu_list) < 1:
            self.__cheapestRouteCost = "No valid route exists"
            self.__cheapestRoute = "No valid route exists"
        # otherwise find cheapest route
        else:
            # generate total route cost for each route permutation and store in cost list
            cost_list = []
            for i in range(len(permu_list)):
                # print(permu_list[i])
                single_flight_cost, route_total_cost = 0, 0

                for j in range(len(permu_list[i]) - 1):
                    # cost of single flight
                    # print(permu_list[i][j], permu_list[i][j+1], single_flight_cost)
                    single_flight_cost = self.__costTable[permu_list[i][j]][permu_list[i][j + 1]]
                    # print(permu_list[i][j], permu_list[i][j+1], single_flight_cost)
                    route_total_cost += single_flight_cost
                # print("Total:", route_total_cost)
                # append the route total cost for the permutation to cost list
                cost_list.append(round(route_total_cost, 2))

            # print("\n",cost_list,"\n")

            # set instance attributes based on the cheapest calculated cost
            cheapest_route = []
            for i in range(len(cost_list)):
                if cost_list[i] == min(cost_list):
                    self.__cheapestRouteCost = cost_list[i]
                    self.__cheapestRoute = permu_list[i]
                    

    def printImpossibleLegs(self):
        """Method to print out all invalid routes if they exist"""

        print(f"2. Impossible journeys - Plane range: {self.__maxRange}km")
        print("======================================================")
        if len(self.__impossibleLegs) < 1:
            print("All journeys are valid")
        else:
            for item in self.__impossibleLegs:
                item = list(item)
                print(f"{item[0]} <---> {item[1]} ---> Distance {self.__distTable[item[0]][item[1]]:6.7}km")
        print("======================================================\n")

        
    def printResult(self,Codes):
        """Method to print out the cheapest route if a vaild route exists"""

        print("1. Route results for input data :", self.__input)
        print("======================================================")
        if len(self.__permutationList) < 1:
            print("There is no valid route")
        else:
            print(f"The cheapest route is {self.__cheapestRoute}.\nThe cheapest route costs €{self.__cheapestRouteCost}.")
            # store the result in class variable (dictionary) key= Codes, value= result
            Route.final[tuple(Codes)]="The cheapest route is "+str(self.__cheapestRoute)+" The cheapest route costs € "+str(self.__cheapestRouteCost)
        print("======================================================\n")
        
        
    def writeResults(self):
        """Method to write results to csv file"""
        
        lines = copy.deepcopy(self.__cheapestRoute)
        if isinstance(lines, list):
            lines.append(str(self.__cheapestRouteCost))
        result = pd.DataFrame([lines])
        result.to_csv("bestroutes.csv", mode='a', index=False, header=False)
        
        
    def getStartAirport(self):
        return self.__startAirport

    
    def getEndAirport(self):
        return self.__endAirport

    
    def getAirportNames(self):
        return self.__airportNames

    
    def getCheapestRoute(self):
        return self.__cheapestRoute

    
    def getCheapestRouteCost(self):
        return self.__cheapestRouteCost

    
    def getMaxRange(self):
        return self.__maxRange

    
    def getPermutationList(self):
        return self.__permutationList

    
    def getImpossibleLegs(self):
        "return impossible legs as a list of tuples"
        
        impossibleLegs = []
        for item in self.__impossibleLegs:
            item = tuple(item)
            impossibleLegs.append(item)
        return impossibleLegs
    
    
    def printReport(self, Codes):
        print("Printing RouteReport")
        print("========================================================================================\n")
        self.printResult(Codes)
        self.printImpossibleLegs()
        self.printDistanceTable()
        self.printCostTable()
        self.printAircraftInformation()
        self.printAirportInformation()
        self.writeResults()
        print("\n")
        

### Run Programme

In [9]:
# instantiate aircraft and airports
aircraftAtlas = AircraftAtlas(aircraft_df, test_df)
atlas = AirportAtlas(airport_df, test_df)
Cache=[]
# for each row in test route calucalte the cheapest route
for i, row in test_df.iterrows():
    Cache.append(row.to_string)
    aircraftType = row[len(row) - 1] #assign the aircraft type, the last element in the row
    routeCodes = []
    
    # append the rest of the elements to a list, the airport codes for the route
    for j in range(len(row)):
        routeCodes.append(row[j])
    test=set(routeCodes)
    count=False
    
    # iterate over stored routes, if key is found print result
    for keys,value in Route.final.items():
        if set(keys)==test:
            print("For Route",keys,"here is the result")
            print(Route.final[keys])
            print()
            count=True
    if count==False:
        AircraftAtlas.getAircraft(aircraftType).setRoute(routeCodes)
        routeCodes.pop()
        for airportCode in routeCodes:
            AirportAtlas.getAirport(airportCode).decreaseOccurances()

Printing RouteReport

1. Route results for input data : ['DUB', 'LHR', 'SYD', 'JFK', 'AAL', '777']
There is no valid route

2. Impossible journeys - Plane range: 15610.21km
SYD <---> DUB ---> Distance 17215.26km
AAL <---> SYD ---> Distance 16140.27km
LHR <---> SYD ---> Distance 17020.32km
SYD <---> JFK ---> Distance 16013.51km

3. Distance Table (km)
          AAL       DUB       JFK       LHR       SYD
AAL      0.00   1096.72   5966.92    913.42  16140.27
DUB   1096.72      0.00   5102.98    448.78  17215.26
JFK   5966.92   5102.98      0.00   5539.49  16013.51
LHR    913.42    448.78   5539.49      0.00  17020.32
SYD  16140.27  17215.26  16013.51  17020.32      0.00

4. Cost Table (€)
         AAL       DUB       JFK       LHR       SYD
AAL     0.00   1096.72   5661.41   1281.44  11706.54
DUB   146.96      0.00   4841.71    629.59  12486.23
JFK   799.57   5102.98      0.00   7771.35  11614.60
LHR   122.40    448.78   5255.87      0.00  12344.84
SYD  2162.80  17215.26  15193.62  23877

### Demonstrate caching
Running the same test file again we show the stored results

In [10]:
# instantiate aircraft and airports
aircraftAtlas = AircraftAtlas(aircraft_df, test_df)
atlas = AirportAtlas(airport_df, test_df)
Cache=[]
# for each row in test route calucalte the cheapest route
for i, row in test_df.iterrows():
    Cache.append(row.to_string)
    aircraftType = row[len(row) - 1] #assign the aircraft type, the last element in the row
    routeCodes = []
    
    # append the rest of the elements to a list, the airport codes for the route
    for j in range(len(row)):
        routeCodes.append(row[j])
    test=set(routeCodes)
    count=False
    
    # iterate over stored routes, if key is found print result
    for keys,value in Route.final.items():
        if set(keys)==test:
            print("For Route",keys,"here is the result")
            print(Route.final[keys])
            print()
            count=True
    if count==False:
        AircraftAtlas.getAircraft(aircraftType).setRoute(routeCodes)
        routeCodes.pop()
        for airportCode in routeCodes:
            AirportAtlas.getAirport(airportCode).decreaseOccurances()

Printing RouteReport

1. Route results for input data : ['DUB', 'LHR', 'SYD', 'JFK', 'AAL', '777']
There is no valid route

2. Impossible journeys - Plane range: 15610.21km
SYD <---> DUB ---> Distance 17215.26km
AAL <---> SYD ---> Distance 16140.27km
LHR <---> SYD ---> Distance 17020.32km
SYD <---> JFK ---> Distance 16013.51km

3. Distance Table (km)
          AAL       DUB       JFK       LHR       SYD
AAL      0.00   1096.72   5966.92    913.42  16140.27
DUB   1096.72      0.00   5102.98    448.78  17215.26
JFK   5966.92   5102.98      0.00   5539.49  16013.51
LHR    913.42    448.78   5539.49      0.00  17020.32
SYD  16140.27  17215.26  16013.51  17020.32      0.00

4. Cost Table (€)
         AAL       DUB       JFK       LHR       SYD
AAL     0.00   1096.72   5661.41   1281.44  11706.54
DUB   146.96      0.00   4841.71    629.59  12486.23
JFK   799.57   5102.98      0.00   7771.35  11614.60
LHR   122.40    448.78   5255.87      0.00  12344.84
SYD  2162.80  17215.26  15193.62  23877

## Unit tests on Route Class

In [11]:
import unittest

# instantiate aircraft and airports
aircraftAtlas = AircraftAtlas(aircraft_df, test_df)
atlas = AirportAtlas(airport_df, test_df)


class TestRoute(unittest.TestCase):
    
    def test_One_Airports_(self):
        t = Route(['DUB', "777"])
        self.assertTrue(t.getCheapestRoute() == ['DUB', 'DUB'])
    
    
    def test_Two_Airports_(self):
        t = Route(['DUB', "LHR", "777"])
        self.assertTrue(t.getCheapestRoute() == ['DUB', 'LHR', 'DUB'])
    
    
    def test_Three_Airports_(self):
        t = Route(['DUB', "LHR", "AAL", "777"])
        self.assertTrue(t.getCheapestRoute() == ['DUB', 'AAL', 'LHR', 'DUB'])
    
    
    def test_Four_Airports_(self):
        t = Route(['DUB', "LHR", "SFO", "AAL", "777"])
        self.assertTrue(t.getCheapestRoute() == ['DUB', 'LHR', 'AAL', 'SFO', 'DUB'])
    
    
    def test_Six_Airports_(self):
        t = Route(['DUB', "LHR","SNN", "ORK", "SFO", "AAL", "777"])
        self.assertTrue(t.getCheapestRoute() == ['DUB', 'LHR', 'AAL', 'SFO', 'SNN', 'ORK', 'DUB'])
        
        
    def test_Seven_Airports_(self):
        t = Route(['DUB', "LHR","SNN", "ORK", "SFO", "CPH", "AAL", "777"])
        self.assertTrue(t.getCheapestRoute() == ['DUB', 'LHR', 'AAL', 'CPH', 'SFO', 'SNN', 'ORK', 'DUB'])
    
    
    def test_Aircraft_Range(self):
        t = Route(['DUB', "LHR", "SYD", "JFK", "AAL", "737"])
        self.assertTrue(t.getMaxRange() == 9012.08)
        
        
    def test_Aircraft_Range2(self):
        t = Route(['DUB', "LHR", "SYD", "JFK", "AAL", "777"])
        self.assertTrue(t.getMaxRange() == 15610.21)
        
        
    def test_Start_Airport(self):
        t = Route(['LHR', "DUB", "SYD", "JFK", "AAL", "737"])
        self.assertTrue(t.getStartAirport() == ['LHR'])
                
            
    def test_End_Airport(self):
        t = Route(['LHR', "DUB", "SYD", "JFK", "AAL", "737"])
        self.assertTrue(t.getEndAirport() == ['LHR'])
        
    
    def test_Airport_Names(self):
        t = Route(['DUB', "LHR", "SYD", "JFK", "AAL", "737"])
        self.assertTrue(t.getAirportNames() == ['Dublin', 
                                           'Heathrow', 
                                           'Sydney Intl', 
                                           'John F Kennedy Intl', 
                                           'Aalborg'])
        
        
    def test_Cheapest_Route(self):
        t = Route(['DUB', "LHR", "SYD", "JFK", "AAL", "737"])  
        self.assertTrue(t.getCheapestRoute() == "No valid route exists")
        
        
    def test_Cheapest_Route2(self):
        t = Route(['DUB', "LHR", "ORK", "SFO", "AAL", "777"])  
        self.assertTrue(t.getCheapestRoute() == ['DUB', 'LHR', 'AAL', 'SFO', 'ORK', 'DUB'])
        
                
unittest.main(argv=[''], verbosity=2, exit=False)

test_Aircraft_Range (__main__.TestRoute) ... ok
test_Aircraft_Range2 (__main__.TestRoute) ... ok
test_Airport_Names (__main__.TestRoute) ... ok
test_Cheapest_Route (__main__.TestRoute) ... ok
test_Cheapest_Route2 (__main__.TestRoute) ... ok
test_End_Airport (__main__.TestRoute) ... ok
test_Four_Airports_ (__main__.TestRoute) ... ok
test_One_Airports_ (__main__.TestRoute) ... ok
test_Seven_Airports_ (__main__.TestRoute) ... ok
test_Six_Airports_ (__main__.TestRoute) ... ok
test_Start_Airport (__main__.TestRoute) ... ok
test_Three_Airports_ (__main__.TestRoute) ... ok
test_Two_Airports_ (__main__.TestRoute) ... ok

----------------------------------------------------------------------
Ran 13 tests in 0.051s

OK
